# Preprocessing

## The initial Excel file downloaded from the Netflix account. 

In [42]:
import pandas as pd

In [9]:
file_path = '/Users/berilenbatan/Downloads/MyNetflixViewingHistory.xlsx'

In [10]:
df = pd.read_excel(file_path, usecols=[0])

In [11]:
df.columns = ["TitleDate"]

In [14]:
df.head()

,TitleDate
0,"A True Gentleman,""11/11/24"""
1,"How to Lose a Guy in 10 Days,""10/24/24"""
2,"Gossip Girl: Season 6: New York, I Love You, X..."
3,"Gossip Girl: Season 6: The Revengers,""10/16/24"""
4,Gossip Girl: Season 6: It's Really Complicated...


## Initially, the WatchDate is split from the coma ',' the Season part is split from ":" ; the quatation marks '"' are stripped. 

In [ ]:
def split_data(row):
    data = row.split(",", 1)  # First coma
    title_part = data[0].strip()
    watch_date = data[1].strip("\"") if len(data) > 1 else None  # Strip the quotation marks
   
    # Sezon ve bölüm bilgisi kontrolü
    if "Season" in title_part:
        parts = title_part.split(":")  
        title = parts[0].strip()
        season = parts[1].strip() if len(parts) > 1 else None
        episode = ":".join(parts[2:]).strip() if len(parts) > 2 else None
    else:
        title = title_part
        season = None
        episode = None

    return title, season, episode, watch_date

# Fonksiyonu tüm satırlara uygula
df = df.dropna(subset=["TitleDate"])  # Boş satırları temizle
df[["Title", "Season", "Episode", "Watch Date"]] = df["TitleDate"].apply(lambda x: pd.Series(split_data(x)))

# Tarih formatını düzeltme
df["Watch Date"] = df["Watch Date"].str.extract(r'(\d{1,2}/\d{1,2}/\d{2,4})')

# Gereksiz "Raw Data" sütununu kaldır
df.drop(columns=["TitleDate"], inplace=True)

# İşlenmiş veriyi yeni bir Excel dosyasına kaydet
processed_file_path = "/Users/berilenbatan/Desktop/Cleaned_Netflix_Viewing_History.xlsx"
df.to_excel(processed_file_path, index=False)

print(f"İşlenmiş dosya kaydedildi: {processed_file_path}")

In [46]:
processed_file_path = "/Users/berilenbatan/Desktop/DSA210/Cleaned_Netflix_Viewing_History.xlsx"
preprocessed1 = pd.read_excel(processed_file_path)
preprocessed1.head(10)

,Title,Season,Episode,Watch Date
0,A True Gentleman,NaN,NaN,11/11/24
1,How to Lose a Guy in 10 Days,NaN,NaN,10/24/24
2,Gossip Girl,Season 6,New York,10/16/24
3,Gossip Girl,Season 6,The Revengers,10/16/24
4,Gossip Girl,Season 6,It's Really Complicated,10/15/24
5,Gossip Girl,Season 6,Save the Last Chance,10/15/24
6,Gossip Girl,Season 6,Where the Vile Things Are,10/15/24
7,Gossip Girl,Season 6,Monstrous Ball,10/15/24
8,Gossip Girl,Season 6,Portrait of a Lady Alexander,10/15/24
9,Gossip Girl,Season 6,Dirty Rotten Scandals,10/14/24


## If the title itself has :, it couldn't seperate the Season part. by only processing the ones with the empty season columns and not touching anything else, the similar operation is done again. 

In [ ]:
import pandas as pd

# Load the cleaned Netflix data
file_path = '/Users/berilenbatan/Desktop/Cleaned_Netflix_Viewing_History.xlsx'
data = pd.read_excel(file_path)

# Function to reprocess rows where "Season" is empty
def extract_season_details(row):
    if pd.isna(row['Season']) and isinstance(row['Title'], str) and "Season" in row['Title']:
        parts = row['Title'].split(":")
        title = parts[0].strip()
        season = parts[1].strip() if len(parts) > 1 else None
        episode = ":".join(parts[2:]).strip() if len(parts) > 2 else None
        return pd.Series([title, season, episode])
    return pd.Series([row['Title'], row['Season'], row['Episode']])

# Apply the function only to rows where "Season" is empty
data[["Title", "Season", "Episode"]] = data.apply(extract_season_details, axis=1)

# Separate movies and TV shows
movies_data = data[data["Season"].isna()]  # Movies have NaN in "Season"
tv_shows_data = data[~data["Season"].isna()]  # TV shows have values in "Season"

# Save the processed data to your Desktop
movies_file_path = "/Users/berilenbatan/Desktop/movies_only_netflix_data.xlsx"
tv_shows_file_path = "/Users/berilenbatan/Desktop/tv_shows_netflix_data.xlsx"
movies_data.to_excel(movies_file_path, index=False)
tv_shows_data.to_excel(tv_shows_file_path, index=False)

print(f"Movies data saved to '{movies_file_path}'.")
print(f"TV shows data saved to '{tv_shows_file_path}'.")


In [47]:
movies2 = "/Users/berilenbatan/Desktop/DSA210/movies_only_netflix_data.xlsx"
preprocessed2_M = pd.read_excel(movies2)
preprocessed2_M.head(10)

,Title,Season,Episode,Watch Date
0,A True Gentleman,NaN,NaN,11/11/24
1,How to Lose a Guy in 10 Days,NaN,NaN,10/24/24
2,Allied,NaN,NaN,6/27/24
3,Queen Charlotte: A Bridgerton Story: Even Days,NaN,NaN,5/25/24
4,Sex and the City: The Movie,NaN,NaN,4/6/24
5,One Day: Limited Series: Episode 7,Limited Series,NaN,2/14/24
6,One Day: Limited Series: Episode 10,Limited Series,NaN,2/14/24
7,One Day: Limited Series: Episode 9,Limited Series,NaN,2/14/24
8,One Day: Limited Series: Episode 8,Limited Series,NaN,2/14/24
9,One Day: Limited Series: Episode 6,Limited Series,NaN,2/14/24


In [48]:
shows2 = "/Users/berilenbatan/Desktop/DSA210/tv_shows_netflix_data.xlsx"
preprocessed2_S = pd.read_excel(shows2)
preprocessed2_S.head(10)

,Title,Season,Episode,Watch Date
0,Gossip Girl,Season 6,New York,10/16/24
1,Gossip Girl,Season 6,The Revengers,10/16/24
2,Gossip Girl,Season 6,It's Really Complicated,10/15/24
3,Gossip Girl,Season 6,Save the Last Chance,10/15/24
4,Gossip Girl,Season 6,Where the Vile Things Are,10/15/24
5,Gossip Girl,Season 6,Monstrous Ball,10/15/24
6,Gossip Girl,Season 6,Portrait of a Lady Alexander,10/15/24
7,Gossip Girl,Season 6,Dirty Rotten Scandals,10/14/24
8,Gossip Girl,Season 6,High Infidelity,10/14/24
9,Gossip Girl,Season 6,Gone Maybe Gone,10/14/24


## Some TV shows call Seasons "Chapter, Part, vol. etc but these shows also name the Episodes differently. "Part" corresponds to episode in some TV show and season in another. There isn't a specific pattern. Stranger Things Season 3 is called Stranger Things 3: ... (episode name) for instance. Also, some shows are called Limited Series and have no Season info, or the V shows that has lasted for one season don't have "Season" keyword in the title. Therefore, I filled the Season column of some TV shows that falled into the movies excel by hand.

## Then, the TV shows that fell into the movies file were seperated based on whether there was a Season column or not. 

In [22]:
# Load the "Movies Only Netflix Data"
file_path = '/Users/berilenbatan/Desktop/movies_only_netflix_data.xlsx'
data = pd.read_excel(file_path)

# Separate movies and TV shows based on the "Season" column
movies_data = data[data["Season"].isna()]  # Movies have NaN in "Season"
tv_shows_data = data[~data["Season"].isna()]  # TV shows have values in "Season"

# Save the processed data to separate files on the desktop
movies_file_path = "/Users/berilenbatan/Desktop/final_movies_data.xlsx"
tv_shows_file_path = "/Users/berilenbatan/Desktop/final_tv_shows_data.xlsx"
movies_data.to_excel(movies_file_path, index=False)
tv_shows_data.to_excel(tv_shows_file_path, index=False)

print(f"Movies data saved to '{movies_file_path}'.")
print(f"TV shows data saved to '{tv_shows_file_path}'.")


Movies data saved to '/Users/berilenbatan/Desktop/final_movies_data.xlsx'.
TV shows data saved to '/Users/berilenbatan/Desktop/final_tv_shows_data.xlsx'.


## The initial TV show Excel file and the one seperated from the initial movies file is merged: 

In [23]:
import pandas as pd

# Load the two TV shows datasets
file_path_1 = '/Users/berilenbatan/Desktop/tv_shows_netflix_data.xlsx'
file_path_2 = '/Users/berilenbatan/Desktop/final_tv_shows_data.xlsx'

tv_shows_data_1 = pd.read_excel(file_path_1)
tv_shows_data_2 = pd.read_excel(file_path_2)

# Merge the datasets
merged_tv_shows_data = pd.concat([tv_shows_data_1, tv_shows_data_2], ignore_index=True)

# Save the merged dataset to a new file
merged_file_path = '/Users/berilenbatan/Desktop/TV_SHOWS_MERGED.xlsx'
merged_tv_shows_data.to_excel(merged_file_path, index=False)

print(f"Merged data saved to '{merged_file_path}'.")


Merged data saved to '/Users/berilenbatan/Desktop/TV_SHOWS_MERGED.xlsx'.


## Season and episode columns are deleted from the Movies excel file. 

## The Final Movies Data's WatchDate is seperated into Month and Year for meaningful findings. 

In [ ]:
import pandas as pd
import calendar

# Load the dataset
file_path = '/Users/berilenbatan/Desktop/final_movies_data.xlsx'   # Replace with the correct path
data = pd.read_excel(file_path)

# Split the "Watch Date" column into Month and Year
data[['Month', 'Year']] = data['Watch Date'].str.split('/', n=2, expand=True)[[0, 2]]

# Convert the Month column from numbers to names
data['Month'] = data['Month'].apply(lambda x: calendar.month_name[int(x)] if pd.notna(x) and x.isdigit() else None)

# Save the updated dataset
output_file_path = '/Users/berilenbatan/Desktop/final_movies_data_with_date.xlsx'  # Replace with the desired output path
data.to_excel(output_file_path, index=False)

print(f"Updated dataset with separated date columns saved to '{output_file_path}'.")


## The WatchDate column is seperated into month and year in the TV shows file as well. 

In [32]:
import pandas as pd
import calendar

# Load the dataset
file_path = '/Users/berilenbatan/Desktop/TV_SHOWS_MERGED.xlsx'  # Replace with the correct path
data = pd.read_excel(file_path)

# Split the "Watch Date" column into Month and Year
data[['Month', 'Year']] = data['Watch Date'].str.split('/', n=2, expand=True)[[0, 2]]

# Convert the Month column from numbers to names
data['Month'] = data['Month'].apply(lambda x: calendar.month_name[int(x)] if pd.notna(x) and x.isdigit() else None)

# Save the updated dataset
output_file_path = "/Users/berilenbatan/Desktop/TV_SHOWS_MERGED_with_date.xlsx"  # Replace with the desired output path
data.to_excel(output_file_path, index=False)

print(f"Updated dataset with separated date columns saved to '{output_file_path}'.")


Updated dataset with separated date columns saved to '/Users/berilenbatan/Desktop/TV_SHOWS_MERGED_with_date.xlsx'.


## The information "ReleaseDate", "IMDb Rating", "Votes", "Director", "ProductionCompany" and "Country is fetched from The Movie Database 3 by 3 to avoid runtime errors. 

In [ ]:
import pandas as pd
import requests

# Load your dataset
file_path = '/Users/berilenbatan/Desktop/final_movies_data_with_date.xlsx'  # Replace with your actual file path
data = pd.read_excel(file_path)

# Define the function to fetch OMDb data
def fetch_omdb_data(title):
    api_key = '4a2df2d0a07d5b3bd499e4ca6e7413a8' 
    cleaned_title = title.split("(")[0].strip()  # Clean the title
    url = f'http://www.omdbapi.com/?t={cleaned_title}&apikey={api_key}'
    response = requests.get(url)
    
    if response.status_code == 200:
        movie_data = response.json()
        print(f"Title: {cleaned_title} | Response: {movie_data}")  # Debugging output
        if movie_data.get("Response") == "True":
            return pd.Series({
                "Genres": movie_data.get("Genre", None),
                "Runtime": movie_data.get("Runtime", None),
                "ReleaseYear": movie_data.get("Year", None),
                "IMDbRating": movie_data.get("imdbRating", None),
                "Votes": movie_data.get("imdbVotes", None),
                "Director": movie_data.get("Director", None),
                "ProductionCompany": movie_data.get("Production", None),
                "Country": movie_data.get("Country", None)
            })
        else:
            print(f"Error fetching data for title '{cleaned_title}': {movie_data.get('Error')}")
    else:
        print(f"HTTP Error for '{cleaned_title}': {response.status_code}")
    
    return pd.Series({
        "ReleaseYear": None,
        "IMDbRating": None,
        "Votes": None,
    })

# Apply the function to fetch OMDb data
print("Fetching OMDb data. This may take a while...")
imdb_data = data['Title'].apply(fetch_omdb_data)

# Merge the OMDb data with your dataset
data = pd.concat([data, imdb_data], axis=1)

# Save the updated dataset
output_file_path = '/Users/berilenbatan/Desktop/movies_with_3info.xlsx' # Replace with your desired output path
data.to_excel(output_file_path, index=False)

print(f"Updated dataset with OMDb data saved to '{output_file_path}'.")


In [38]:
import requests
import pandas as pd

# Load your dataset
file_path = '/Users/berilenbatan/Desktop/movies_with_3info.xlsx'  # Replace with your actual file path
data = pd.read_excel(file_path)

# Define your TMDb API key
tmdb_api_key = '4a2df2d0a07d5b3bd499e4ca6e7413a8'  # Replace with your TMDb API key

# Function to fetch movie details from TMDb
def fetch_tmdb_data(title):
    base_url = "https://api.themoviedb.org/3"
    search_url = f"{base_url}/search/movie"
    details_url_template = f"{base_url}/movie/{{movie_id}}"
    credits_url_template = f"{base_url}/movie/{{movie_id}}/credits"

    try:
        # Search for the movie
        search_params = {"api_key": tmdb_api_key, "query": title}
        search_response = requests.get(search_url, params=search_params)
        if search_response.status_code == 200:
            search_results = search_response.json().get("results", [])
            if search_results:
                # Get the first movie's TMDb ID
                movie_id = search_results[0]["id"]

                # Fetch additional movie details
                details_response = requests.get(details_url_template.format(movie_id=movie_id), params={"api_key": tmdb_api_key})
                details = details_response.json() if details_response.status_code == 200 else {}

                # Fetch credits to get the director
                credits_response = requests.get(credits_url_template.format(movie_id=movie_id), params={"api_key": tmdb_api_key})
                credits = credits_response.json() if credits_response.status_code == 200 else {}
                director = next((crew["name"] for crew in credits.get("crew", []) if crew["job"] == "Director"), None)

                # Return all relevant data
                return pd.Series({
                    "Director": director,
                    "ProductionCompany": ", ".join(pc["name"] for pc in details.get("production_companies", [])),
                    "Country": ", ".join(c["name"] for c in details.get("production_countries", []))
                })
    except Exception as e:
        print(f"Error fetching data for '{title}': {e}")
    
    # Return None for all fields if data fetch fails
    return pd.Series({
        "Director": None,
        "ProductionCompany": None,
        "Country": None
    })

# Apply the function to fetch TMDb data
print("Fetching TMDb data. This may take a while...")
tmdb_data = data['Title'].apply(fetch_tmdb_data)

# Merge the TMDb data with your dataset
data = pd.concat([data, tmdb_data], axis=1)

# Save the updated dataset
output_file_path = '/Users/berilenbatan/Desktop/movies_with_5info.xlsx'  # Replace with your desired output path
data.to_excel(output_file_path, index=False)

print(f"Updated dataset with TMDb data saved to '{output_file_path}'.")



Fetching TMDb data. This may take a while...
Updated dataset with TMDb data saved to '/Users/berilenbatan/Desktop/movies_with_5info.xlsx'.
